In [1]:
#!/usr/bin/env python
# -*- mode:python; coding:utf-8 -*-

import os, sys, inspect
current_folder = os.path.realpath(os.path.abspath(os.path.split(inspect.getfile(inspect.currentframe()))[0]))
folder_parts = current_folder.split(os.sep)
pywikibot_folder = os.sep.join(folder_parts[0:-1])
if current_folder not in sys.path:
    sys.path.insert(0, current_folder)
if pywikibot_folder not in sys.path:
    sys.path.insert(0, pywikibot_folder)

import pywikibot as pb
from pywikibot.specialbots import UploadRobot

from bs4 import BeautifulSoup
import urllib2
from mako.template import Template
import re
import calendar
from openpyxl import Workbook

In [22]:
import urllib2
from PIL import ImageFile

def getsizes(uri):
    # get file size *and* image size (None if not known)
    href = urllib2.quote(uri.encode('utf-8'), ':/')
    try:
        img_request = urllib2.Request(href, headers=headers)
        file = urllib2.urlopen(img_request)
    except :
        return 0, None
    size = file.headers.get("content-length")
    if size:
        size = int(size)
    p = ImageFile.Parser()
    while 1:
        data = file.read(1024)
        if not data:
            break
        p.feed(data)
        if p.image:
            return size, p.image.size
            break
    file.close()
    return size, None

def sanitize_filename (filename):
    filename = filename.replace(': ', ' - ').replace('  ', '')
    quote_chars = ['?', '<', '>', '"', '|']
    space_chars = ['/', '\\', '*', '|']
    chars = list(filename)
    chars = [c if c not in quote_chars else "'" for c in chars]
    chars = [c if c not in space_chars else " " for c in chars]
    return ''.join(chars)    

In [23]:
commons_site = pb.Site("commons", "commons")
wb = Workbook()
ws = wb.active
EXCEL_FILE = "diario.madrid.es.xlsx"

In [24]:
'''url = 'https://diario.madrid.es/blog/notas-de-prensa/siete-lineas-de-autobuses-refuerzan-su-servicio/'
user_agent = 'Mozilla/5.0 (Windows NT 6.1; WOW64; rv:40.0) Gecko/20100101 Firefox/40.1'
headers = { 'User-Agent' : user_agent }

req = urllib2.Request(url, headers=headers)
response = urllib2.urlopen(req).read()
soup = BeautifulSoup(response, 'html.parser')
images = soup.find_all(attrs={'class': re.compile(r"wp-image-\d*")})
for image in images :
    links = {int(x.strip().split(' ')[1][:-1]): x.strip().split(' ')[0] for x in image.get('srcset').split(',')}
    sortedlist = [(k, links[k]) for k in sorted(links)]
    if getsizes(image.parent.get('href'))[0] > getsizes(sortedlist[-1][1])[0] :
        print image.parent.get('href')
    else :
        print sortedlist[-1][1]'''

'url = \'https://diario.madrid.es/blog/notas-de-prensa/siete-lineas-de-autobuses-refuerzan-su-servicio/\'\nuser_agent = \'Mozilla/5.0 (Windows NT 6.1; WOW64; rv:40.0) Gecko/20100101 Firefox/40.1\'\nheaders = { \'User-Agent\' : user_agent }\n\nreq = urllib2.Request(url, headers=headers)\nresponse = urllib2.urlopen(req).read()\nsoup = BeautifulSoup(response, \'html.parser\')\nimages = soup.find_all(attrs={\'class\': re.compile(r"wp-image-\\d*")})\nfor image in images :\n    links = {int(x.strip().split(\' \')[1][:-1]): x.strip().split(\' \')[0] for x in image.get(\'srcset\').split(\',\')}\n    sortedlist = [(k, links[k]) for k in sorted(links)]\n    if getsizes(image.parent.get(\'href\'))[0] > getsizes(sortedlist[-1][1])[0] :\n        print image.parent.get(\'href\')\n    else :\n        print sortedlist[-1][1]'

In [27]:
user_agent = 'Mozilla/5.0 (Windows NT 6.1; Win64; x64)'
headers = {'User-Agent': user_agent}
url_base = 'https://diario.madrid.es/blog/notas-de-prensa/'

req = urllib2.Request(url_base, headers=headers)
response = urllib2.urlopen(req)
r = response.read()
soup = BeautifulSoup(r, "html.parser")
pages = soup.find_all("a", class_="page-numbers")
final_page = int(pages[-2].get_text())-1

row = []
for i in range (final_page) :
    if i == 0 :
        url = url_base
    elif i > 15 :
        break
    else :
        url = u'{0}page/{1}'.format(url_base, i + 1)
    req = urllib2.Request(url, headers=headers)
    response = urllib2.urlopen(req)
    r = response.read()
    soup = BeautifulSoup(r)
    
    articles = soup.find_all("h2", class_="post-title")
    for article in articles :
        url = article.a["href"]
        #print '#', url
        req = urllib2.Request(url, headers=headers)
        response = urllib2.urlopen(req).read()
        soup = BeautifulSoup(response, 'html.parser')

        title = soup.find_all("h1", class_="post-title")[0].get_text().strip().replace('  ', ' ')
        date = '-'.join(soup.find_all("div", class_="post-date")[0].get_text().strip().split('/')[::-1])
        year = date.split('-')[0]
        month = calendar.month_name[int(date.split('-')[1])]
        body = soup.find_all("div", class_="post-content")
        p_description = body[0].find_all("p")
        #print p_description
        for p in p_description :
            if len(p.get_text()) > 10:
                description = p.get_text()
                break

        counter = 0
        for i, image in enumerate(soup.find_all("div", class_="gallery-icon")) :
            try:
                href = urllib2.quote(image.a["href"].encode('utf-8'), ':/')
            except :
                continue
            print href
            image_name = u'%s (%02d).jpg' % (title, counter+1)
            image_name = sanitize_filename(image_name)
            #image_path = r'D:\\temp\\' + image_url.split('/')[-1]
            image_path = r'D:\\temp\\' + image_name
            print image_path
            try:
                img_request = urllib2.Request(href, headers=headers)
                img_data = urllib2.urlopen(img_request).read()
                f = open(image_path, 'wb')
                f.write(img_data)
                f.close()
                row = [image_path, href, title, url, date, year, month, description]
                ws.append(row)
            except :
                print 'Failed download. Skipping'
            counter += 1

        images = soup.find_all(attrs={'class': re.compile(r"wp-image-\d*")})
        for i, image in enumerate(images) :
            try:
                links = {int(x.strip().split(' ')[1][:-1]): x.strip().split(' ')[0] for x in image.get('srcset').split(',')}
                sortedlist = [(k, links[k]) for k in sorted(links)]
                link = sortedlist[-1][1]
            except :
                link = image.get('src')
            if image.parent.get('href') == None :
                image_url = sortedlist[-1][1]
                image_url = urllib2.quote(image_url.encode('utf-8'), ':/')
                print sortedlist[-1][1]
            elif getsizes(image.parent.get('href'))[0] > getsizes(link)[0] :
                image_url = image.parent.get('href')
                image_url = urllib2.quote(image_url.encode('utf-8'), ':/')
                print image.parent.get('href')
            else :
                image_url = sortedlist[-1][1]
                image_url = urllib2.quote(image_url.encode('utf-8'), ':/')
                print sortedlist[-1][1]
            image_name = u'%s (%02d).jpg' % (title, counter+i+1)
            image_name = sanitize_filename(image_name)
            image_path = r'D:\\temp\\' + image_name
            print image_path
            try:
                img_request = urllib2.Request(image_url, headers=headers)
                img_data = urllib2.urlopen(img_request).read()
                #image_path = r'D:\\temp\\' + image_url.split('/')[-1]
                f = open(image_path, 'wb')
                f.write(img_data)
                f.close()
                row = [image_path, image_url, title, url, date, year, month, description]
                ws.append(row)
            except :
                print 'Failed download. Skipping'

https://diario.madrid.es/wp-content/uploads/2017/10/DSC_1683-bd-1338x1000.jpg
D:\\temp\\Refuerzos en 55 líneas de la EMT ante el escenario de alta contaminación (01).jpg
https://diario.madrid.es/wp-content/uploads/2017/10/057A0995.jpg
D:\\temp\\Las letras toman Madrid, la I Feria del Libro de Otoño, el Día de las Bibliotecas y ‘Poemad’ (01).jpg
https://diario.madrid.es/wp-content/uploads/2017/10/martirio_copy-jesus-ugalde-452x452.jpg
D:\\temp\\Las letras toman Madrid, la I Feria del Libro de Otoño, el Día de las Bibliotecas y ‘Poemad’ (02).jpg
https://diario.madrid.es/wp-content/uploads/2017/10/Circo.de_.pulgas_0.jpeg
D:\\temp\\Las letras toman Madrid, la I Feria del Libro de Otoño, el Día de las Bibliotecas y ‘Poemad’ (03).jpg
https://diario.madrid.es/wp-content/uploads/2017/10/Danzaeñe-en-El-Español.jpg
D:\\temp\\Las letras toman Madrid, la I Feria del Libro de Otoño, el Día de las Bibliotecas y ‘Poemad’ (04).jpg
https://diario.madrid.es/wp-content/uploads/2017/10/Danzaeñe.jpg
D:\\te

In [28]:
wb.save(EXCEL_FILE)